In [4]:
import arcpy

# 取得目前開啟的 ArcGIS Pro 專案
aprx = arcpy.mp.ArcGISProject("CURRENT")

print("專案路徑：", aprx.filePath)              # .aprx 檔案完整路徑
print("預設 gdb：", aprx.defaultGeodatabase)   # 預設的 geodatabase
print("專案資料夾：", aprx.homeFolder)         # 專案所在的資料夾 (home folder)

# 取得目前活動的 Map (通常就是你 ArcGIS Pro 視窗左邊的那個)
m = aprx.activeMap

print("目前 Map 名稱:", m.name)

sr = m.spatialReference
print("圖層空間參考:", sr.name, sr.factoryCode)

專案路徑： D:\ArcGIS\AI_training\AI_training.aprx
預設 gdb： D:\ArcGIS\AI_training\AI_training.gdb
專案資料夾： D:\ArcGIS\AI_training
目前 Map 名稱: Lizard
圖層空間參考: Transverse_Mercator 0


In [5]:
####分割orthomosaic

import arcpy, os

# ======== 使用者需設定的參數 ========
ortho_path = r"D:\SfM_coral_reefs_product\Lizard_island\LIRS_competition_t1_DMW\Orthos\north_reef_1_t1_ortho.tif"   # 正射影像（raster）路徑或名稱（在地理資料庫內也可）
grid_fc    = "north_reef1_grid_2m"     # 多邊形網格 Feature Class
zone_field = "Zone"                                # 代表分區編號的欄位名稱
out_subdir = "ortho_splits"                        # 會在影像所在資料夾底下建立此子資料夾
# ===================================

arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False  # 自動加到地圖

# 以影像檔的父層資料夾為基準建立輸出資料夾
ortho_dir = arcpy.management.GetRasterProperties(ortho_path, "VALUETYPE")  # 檢查一下確定是 raster
base_dir = os.path.dirname(arcpy.Describe(ortho_path).catalogPath)
out_dir = os.path.join(base_dir, out_subdir)
os.makedirs(out_dir, exist_ok=True)

# 對齊/解析度設定，避免像素位移
arcpy.env.snapRaster = ortho_path
arcpy.env.cellSize   = ortho_path
arcpy.env.outputCoordinateSystem = arcpy.Describe(ortho_path).spatialReference

# 取得 OID 欄位名稱，建立圖層以便選取單一多邊形
oid_field = arcpy.Describe(grid_fc).OIDFieldName
grid_lyr = arcpy.management.MakeFeatureLayer(grid_fc, "grid_lyr")

# 預先算出補零寬度（以 Zone 最大位數與至少 2 位為準）
def zone_to_code(z):
    try:
        # 數字 → 兩位數（或更多，依實際需要自動擴張）
        n = int(z)
        return f"{n:02d}"
    except:
        # 非數字 → 直接用字串
        return str(z)

# 逐格裁切
with arcpy.da.SearchCursor(grid_fc, [oid_field, zone_field]) as cursor:
    for oid, zone_val in cursor:
        if zone_val is None:
            # 無 Zone 值就跳過
            continue

        zone_code = zone_to_code(zone_val)
        out_name  = f"ortho_split_{zone_code}.tif"
        out_path  = os.path.join(out_dir, out_name)

        # 避免重覆處理
        if arcpy.Exists(out_path):
            arcpy.AddMessage(f"已存在，略過：{out_name}")
            continue

        # 選取當前多邊形
        where = f"{arcpy.AddFieldDelimiters(grid_fc, oid_field)} = {oid}"
        arcpy.management.SelectLayerByAttribute(grid_lyr, "NEW_SELECTION", where)

        # 以「ClippingGeometry」裁切，維持外框
        # Clip_management(in_raster, rectangle, out_raster, in_template_dataset, nodata_value, clipping_geometry, maintain_clipping_extent)
        arcpy.management.Clip(
            in_raster=ortho_path,
            rectangle="#",
            out_raster=out_path,
            in_template_dataset=grid_lyr,
            nodata_value="0",
            clipping_geometry="ClippingGeometry",
            maintain_clipping_extent="MAINTAIN_EXTENT"
        )

        arcpy.AddMessage(f"完成：{out_name}")

# 清理選取
arcpy.management.SelectLayerByAttribute(grid_lyr, "CLEAR_SELECTION")
# 移除暫存圖層
arcpy.management.Delete(grid_lyr)

print(f"全部完成！輸出位置：{out_dir}")


全部完成！輸出位置：D:\SfM_coral_reefs_product\Lizard_island\LIRS_competition_t1_DMW\Orthos\ortho_splits


In [6]:
#####分割點圖徵


import arcpy, os

points_fc = "lizard_rr_annotations_2023_north_reef_1_merged"
grid_fc   = "north_reef1_grid_2m"
zone_field = "Zone"
arcpy.env.workspace = r"D:\ArcGIS\AI_training\split_points.gdb"

arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False  # 自動加到地圖

# 先清空舊有的分割結果（避免 T1, T2… 累積）
for fc in arcpy.ListFeatureClasses():
    arcpy.management.Delete(fc)
    print(f"刪除舊的圖徵: {fc}")

# 1) 空間連接
sj_fc = os.path.join(arcpy.env.workspace, "points_with_zone")
if arcpy.Exists(sj_fc):
    arcpy.management.Delete(sj_fc)

arcpy.analysis.SpatialJoin(
    target_features=points_fc,
    join_features=grid_fc,
    out_feature_class=sj_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
    match_option="INTERSECT"
)

# 2) 分拆
arcpy.analysis.SplitByAttributes(sj_fc, arcpy.env.workspace, zone_field)

# 3) 刪掉中間成果
if arcpy.Exists(sj_fc):
    arcpy.management.Delete(sj_fc)
    print(f"已刪除中間圖層: {sj_fc}")

print("完成！結果已存放在:", arcpy.env.workspace)

刪除舊的圖徵: T1
刪除舊的圖徵: T2
刪除舊的圖徵: T3
刪除舊的圖徵: T4
刪除舊的圖徵: T5
刪除舊的圖徵: T6
刪除舊的圖徵: T7
刪除舊的圖徵: T8
刪除舊的圖徵: T9
刪除舊的圖徵: T10
刪除舊的圖徵: T11
刪除舊的圖徵: T12
刪除舊的圖徵: T13
刪除舊的圖徵: T14
刪除舊的圖徵: T15
刪除舊的圖徵: T16
已刪除中間圖層: D:\ArcGIS\AI_training\split_points.gdb\points_with_zone
完成！結果已存放在: D:\ArcGIS\AI_training\split_points.gdb
